# Instantiating the VGG16 convolutional base

In [ ]:
from keras.applications import VGG16
model = VGG16(weights = 'imagenet',
              include_top = False)
model.summary()

# Build a feature extraction model

In [3]:
import keras
layer_name = 'block1_conv1'
layer_output = model.get_layer(layer_name)
feature_extractor = keras.Model(inputs = model.inputs, outputs = layer_output.output)

# Set up the gradient ascent process

In [4]:
import tensorflow as tf
def compute_loss(input_image, filter_index):
  activation = feature_extractor(input_image)
  filter_activation = activation[:,2:-2, 2:2, filter_index]
  return tf.reduce_mean(filter_activation)

In [5]:
@tf.function
def gradient_ascent_step(img, filter_index, learning_rate):
  with tf.GradientTape() as tape:
    tape.watch(img)
    loss = compute_loss(img, filter_index)

  grads = tape.gradient(loss, img)
  grads = tf.math.l2_normalize(grads)
  img += learning_rate * grads
  return loss, img

# Set up the end to end filter visualization loop

In [6]:
import numpy as np
img_width = 180
img_height = 180
def initialize_image():
  img = tf.random.uniform((1, img_width, img_height,3))
  return (img - 0.5) * 0.25

def visualize_filter(filter_index):
  iterations = 30
  learning_rate = 0.01
  img = initialize_image()
  for iteration in range(iterations):
    loss, img = gradient_ascent_step(img, filter_index, learning_rate)

  img = deprocess_image(img[0].numpy())
  return loss, img

def deprocess_image(img):
  img -= img.mean()
  img /= img.std() + 1e-5
  img *= 0.15

  img = img[25:-25, 25:-25,:]

  img += 0.5
  img = np.clip(img, 0,1)

  img *= 255
  img = np.clip(img, 0, 255).astype('uint8')
  return img

In [ ]:
from IPython.display import Image, display

loss, img = visualize_filter(0)
keras.utils.save_img('0.png', img)
display(Image('0.png'))

# Visualize the first 64 filters in the target layer

In [ ]:
all_imgs = []
for filter_index in range(64):
  print('Processing filter %d' %(filter_index))
  loss, img =visualize_filter(filter_index)
  all_imgs.append(img)

margin = 5
n = 8
cropped_width = img_width - 25 * 2
cropped_height = img_height - 25 * 2
width = n * cropped_width + (n - 1) * margin
height = n * cropped_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

for i in range(n):
  for j in range(n):
    img = all_imgs[i * n + j]
    stitched_filters[
        (cropped_width + margin) * i : (cropped_width + margin) * i + cropped_width,
        (cropped_height + margin) * j : (cropped_height + margin) * j + cropped_height,:] = img

keras.utils.save_img('stitched_filters.png', stitched_filters)

from IPython.display import Image, display
display(Image('stitched_filters.png'))